In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())
    

cuda:1


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/2D Klein Gordon/stan'

In [4]:
# !pip install smt

In [5]:
def true_2D_1(xt): #True function for 2D_1 Klein Gordon Equation x \in [-50,50] , t \in [0,10]
    y = xt[:,0]*np.cos(xt[:,1])
    return y.reshape(-1,1)    

In [6]:
label = "KG_atanh"
loss_thresh = 0.01

x = np.linspace(-5,5,500).reshape(-1,1)
t = np.linspace(0,10,1000).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

y_true = true_2D_1(xt)
y_true_norm = np.linalg.norm(y_true,2)

#bound_pts_idx = ((X == -5) + (X == 5) + (T == 0)).reshape(-1,)

#xt_bound = xt[bound_pts_idx,:]
#y_bound = y_true[bound_pts_idx,:]


xt_test_tensor = torch.from_numpy(xt).float().to(device)


lb_xt = xt[0]
ub_xt = xt[-1]

In [7]:
def trainingdata(N_I,N_B,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    x_BC1 = np.random.uniform(size = N_I).reshape(-1,1)
    t_BC1 = np.zeros((N_I,1))
    samples = np.hstack((x_BC1,t_BC1))
    xt_BC1 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC1 = true_2D_1(xt_BC1)
    
    x_BC2 = np.zeros((int(N_B/2),1))
    t_BC2 = np.random.uniform(size = int(N_B/2)).reshape(-1,1)
    samples = np.hstack((x_BC2,t_BC2))
    xt_BC2 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC2 = true_2D_1(xt_BC2)
    
    x_BC3 = np.ones((int(N_B/2),1))
    t_BC3 = np.random.uniform(size = int(N_B/2)).reshape(-1,1)
    samples = np.hstack((x_BC3,t_BC3))
    xt_BC3 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC3 = true_2D_1(xt_BC3)

    xt_BC = np.vstack((xt_BC1,xt_BC2,xt_BC3))
    y_BC = np.vstack((y_BC1,y_BC2,y_BC3))

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll, xt_BC)) # append training points to collocation points 

    return xt_coll, xt_BC, y_BC

In [8]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,n_val):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            
        
        self.alpha = Parameter(torch.ones((50,len(layers)-2)))
        self.alpha.requiresGrad = True
        
        self.n = torch.tensor(n_val)

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.n*self.alpha[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC(self,xt,y):
                
        loss_bc = self.loss_function(self.forward(xt), y)
                
        return loss_bc
    
    def loss_PDE(self, xt_coll, f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        y = self.forward(g) 
        
        
        y_x_t = autograd.grad(y,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        y_xx_tt = autograd.grad(y_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        #du_dx = u_x_t[:,[0]]
        
        d2y_dx2 = y_xx_tt[:,[0]]
        d2y_dt2 = y_xx_tt[:,[1]]    
        

        f = d2y_dt2 - d2y_dx2 + torch.pow(y,2) + (g[:,0]*torch.cos(g[:,1])).reshape(-1,1) - (torch.pow(g[:,0],2)*torch.pow(torch.cos(g[:,1]),2)).reshape(-1,1)
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_BC,y_BC,xt_coll,f_hat):

        loss_BC = self.loss_BC(xt_BC,y_BC)
        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_val = loss_BC + loss_f
        
        return loss_val
         
    'test neural network'
    def test(self):
        y_pred = self.forward(xt_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()
   
        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re  

In [9]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [10]:
def train_step(xt_BC, y_BC, xt_coll, f_hat,seed):
    # x_coll_np_array = colloc_pts(N_f,seed*123)
    # x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
    # f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_BC, y_BC, xt_coll,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        return loss

    optimizer.step(closure)

In [11]:
def train_model(max_iter,rep): 
  print(rep) 
  torch.manual_seed(rep*11)
  start_time = time.time() 
  thresh_flag = 0

  xt_coll, xt_BC, y_BC = trainingdata(N_I,N_B,N_f,rep*11)
  xt_coll = torch.from_numpy(xt_coll).float().to(device)
  xt_BC = torch.from_numpy(xt_BC).float().to(device)
  y_BC = torch.from_numpy(y_BC).float().to(device)

  f_hat = torch.zeros(xt_coll.shape[0],1).to(device)
 
  for i in range(max_iter):
    train_step(xt_BC, y_BC, xt_coll,f_hat,i)
    
    loss_np = PINN.loss(xt_BC, y_BC, xt_coll,f_hat).cpu().detach().numpy()
    if(thresh_flag == 0):
        if(loss_np < loss_thresh):
            time_threshold[rep] = time.time() - start_time
            epoch_threshold[rep] = i+1            
            thresh_flag = 1       
    data_update(loss_np)
    
    print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
    


  elapsed_time[rep] = time.time() - start_time  
  print('Training time: %.2f' % (elapsed_time[rep]))

In [12]:
#for tune_reps in range(25):
  
max_reps = 10 #10
max_iter = 100 #100

train_loss_full = []
test_mse_full = []
test_re_full = []
alpha_full = []
elapsed_time= np.zeros((max_reps,1))
time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

N_I = 200  #Total number of data points for 'y'
N_B = 400
N_f = 10000 #Total number of collocation points
n_val = 1.0  

for reps in range(max_reps):
  print(label)
  print(reps)

  train_loss = []
  test_mse_loss = []
  test_re_loss = []
  alpha_val = []

  torch.manual_seed(reps*36)

  layers = np.array([2,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

  PINN = Sequentialmodel(layers,n_val)

  PINN.to(device)

  'Neural Network Summary'
  print(PINN)

  params = list(PINN.parameters())


  optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.25, 
                            max_iter = 20, 
                            max_eval = 30, 
                            tolerance_grad = 1e-8, 
                            tolerance_change = 1e-8, 
                            history_size = 100, 
                            line_search_fn = 'strong_wolfe')




  nan_flag = train_model(max_iter,reps)

  torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
  train_loss_full.append(train_loss)
  test_mse_full.append(test_mse_loss)
  test_re_full.append(test_re_loss)
  #elapsed_time[reps] = time.time() - start_time
  alpha_full.append(alpha_val)  


  #print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "alpha": alpha_full, "label": label}
savemat(label+'.mat', mdic)

KG_atanh
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 56.616932 Test MSE 7.951808284558346 Test RE 1.3478482859691505
1 Train Loss 45.676773 Test MSE 8.089870267254938 Test RE 1.359498832126889
2 Train Loss 37.604042 Test MSE 7.597744013822945 Test RE 1.317499286841015
3 Train Loss 33.48955 Test MSE 6.608636222178596 Test RE 1.2

95 Train Loss 0.010175719 Test MSE 0.00206056938187433 Test RE 0.021697099541836358
96 Train Loss 0.009654796 Test MSE 0.0017844389717618153 Test RE 0.02019105027032558
97 Train Loss 0.00900599 Test MSE 0.0016967827972814943 Test RE 0.019688887909677894
98 Train Loss 0.008304529 Test MSE 0.0015525647162219459 Test RE 0.01883358152505673
99 Train Loss 0.007865839 Test MSE 0.001581546197847251 Test RE 0.019008550524342072
Training time: 138.71
KG_atanh
1
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50

90 Train Loss 0.04603374 Test MSE 0.03143641581467177 Test RE 0.08474704668557681
91 Train Loss 0.04361316 Test MSE 0.030973255268526685 Test RE 0.08412043042163028
92 Train Loss 0.04188542 Test MSE 0.028245331014629203 Test RE 0.08033067100160744
93 Train Loss 0.04052292 Test MSE 0.02864302959534989 Test RE 0.08089422825964399
94 Train Loss 0.03815218 Test MSE 0.026736026627347927 Test RE 0.0781549505829246
95 Train Loss 0.0365456 Test MSE 0.027362013053319974 Test RE 0.07906460091732394
96 Train Loss 0.035243735 Test MSE 0.026529155326703645 Test RE 0.07785199956309097
97 Train Loss 0.033817936 Test MSE 0.026114906519825738 Test RE 0.07724178440043247
98 Train Loss 0.0327215 Test MSE 0.026061321361949642 Test RE 0.07716249752649242
99 Train Loss 0.031041233 Test MSE 0.022333116253307108 Test RE 0.07143034103102627
Training time: 139.53
KG_atanh
2
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=5

81 Train Loss 0.0013498554 Test MSE 0.0010333139186945127 Test RE 0.01536470406861486
82 Train Loss 0.0013122716 Test MSE 0.0010275500422669047 Test RE 0.015321791601252873
83 Train Loss 0.001280633 Test MSE 0.0010238929840327203 Test RE 0.01529450211367026
84 Train Loss 0.0012569515 Test MSE 0.000994465523234422 Test RE 0.01507311199229345
85 Train Loss 0.0012201393 Test MSE 0.0009620517422393239 Test RE 0.014825429211425422
86 Train Loss 0.001174265 Test MSE 0.0009727492601290731 Test RE 0.014907626897421835
87 Train Loss 0.0011089905 Test MSE 0.0009302662705613774 Test RE 0.014578461612472535
88 Train Loss 0.0010798182 Test MSE 0.0009410557461358898 Test RE 0.014662760324593793
89 Train Loss 0.0010417526 Test MSE 0.0009569018938839257 Test RE 0.014785695818383965
90 Train Loss 0.0010154345 Test MSE 0.0009327029906216148 Test RE 0.0145975423847928
91 Train Loss 0.0009916255 Test MSE 0.000943353410642791 Test RE 0.01468064957551767
92 Train Loss 0.00096701935 Test MSE 0.00087041566375

75 Train Loss 0.5774001 Test MSE 3.427222573605516 Test RE 0.8848694779371994
76 Train Loss 0.57006264 Test MSE 3.4620228298446145 Test RE 0.8893506442431125
77 Train Loss 0.5598219 Test MSE 3.4683003038927565 Test RE 0.8901565816073969
78 Train Loss 0.5496231 Test MSE 3.439241117865323 Test RE 0.8864196450962774
79 Train Loss 0.5384375 Test MSE 3.4119192499201674 Test RE 0.8828916965128969
80 Train Loss 0.5278741 Test MSE 3.3882941796068833 Test RE 0.8798296939092988
81 Train Loss 0.520037 Test MSE 3.386334885084398 Test RE 0.8795752746225128
82 Train Loss 0.5135556 Test MSE 3.383938499522172 Test RE 0.8792639978888052
83 Train Loss 0.5035595 Test MSE 3.35729982688351 Test RE 0.8757963367585307
84 Train Loss 0.49595082 Test MSE 3.3475437302523163 Test RE 0.8745229071421518
85 Train Loss 0.4864627 Test MSE 3.3483202508750627 Test RE 0.8746243316119069
86 Train Loss 0.47787282 Test MSE 3.344951608393537 Test RE 0.8741842545171612
87 Train Loss 0.47154343 Test MSE 3.3419150137165254 Test

...

66 Train Loss 16.461266 Test MSE 8.079364959525261 Test RE 1.3586158394039223
67 Train Loss 16.399452 Test MSE 8.139651358696597 Test RE 1.3636752613693426
68 Train Loss 16.194561 Test MSE 8.166180338333028 Test RE 1.3658957178964897
69 Train Loss 16.115477 Test MSE 8.12437028784137 Test RE 1.362394604073472
70 Train Loss 16.025106 Test MSE 8.12450601754469 Test RE 1.3624059844412137
71 Train Loss 15.96003 Test MSE 8.13634846185885 Test RE 1.3633985581349244
72 Train Loss 15.925512 Test MSE 8.143313593827935 Test RE 1.3639820028754144
73 Train Loss 15.855818 Test MSE 8.118696794819098 Test RE 1.3619188203561254
74 Train Loss 15.787106 Test MSE 8.140421477293994 Test RE 1.3637397706949872
75 Train Loss 15.749645 Test MSE 8.112530527987223 Test RE 1.3614015236606318
76 Train Loss 15.68391 Test MSE 8.128828167498304 Test RE 1.3627683289333126
77 Train Loss 15.642542 Test MSE 8.07849445128375 Test RE 1.3585426456473637
78 Train Loss 15.583764 Test MSE 8.08547518191602 Test RE 1.35912948595

60 Train Loss 0.03971908 Test MSE 0.007397993203523836 Test RE 0.04111166106694133
61 Train Loss 0.03803417 Test MSE 0.006810087567132911 Test RE 0.039444314095059094
62 Train Loss 0.03664517 Test MSE 0.006418933286422862 Test RE 0.03829477227419437
63 Train Loss 0.03529134 Test MSE 0.006362764028036167 Test RE 0.0381268537779755
64 Train Loss 0.03359552 Test MSE 0.007036172664219207 Test RE 0.04009371550670862
65 Train Loss 0.03114381 Test MSE 0.0066877076002521164 Test RE 0.03908829241944145
66 Train Loss 0.029260002 Test MSE 0.006514663978853678 Test RE 0.03857927574576266
67 Train Loss 0.027682612 Test MSE 0.005492009328146144 Test RE 0.035422050133620986
68 Train Loss 0.02629913 Test MSE 0.004659245972051009 Test RE 0.03262615382801233
69 Train Loss 0.02537442 Test MSE 0.004773247548543256 Test RE 0.033022887070739124
70 Train Loss 0.024509229 Test MSE 0.0041713140352303965 Test RE 0.030870559706270822
71 Train Loss 0.022282638 Test MSE 0.003762350149289525 Test RE 0.0293182244950

53 Train Loss 0.13496569 Test MSE 0.019311611488480554 Test RE 0.06642282128089112
54 Train Loss 0.10168146 Test MSE 0.009401692636250722 Test RE 0.04634587487233857
55 Train Loss 0.08552521 Test MSE 0.0077891456572979885 Test RE 0.04218450650645139
56 Train Loss 0.07026392 Test MSE 0.0069579934874039635 Test RE 0.03987035193943151
57 Train Loss 0.06155081 Test MSE 0.00634860148399827 Test RE 0.03808439785373353
58 Train Loss 0.051941633 Test MSE 0.005479943019378116 Test RE 0.03538311644007625
59 Train Loss 0.04329242 Test MSE 0.005687859333195007 Test RE 0.03604810869711674
60 Train Loss 0.037262112 Test MSE 0.004586853009372961 Test RE 0.03237169737343535
61 Train Loss 0.033794556 Test MSE 0.005300670003236712 Test RE 0.03479953526696867
62 Train Loss 0.031844065 Test MSE 0.005615620141733189 Test RE 0.035818461007147064
63 Train Loss 0.027390296 Test MSE 0.004116988385706068 Test RE 0.030668877525963027
64 Train Loss 0.02415821 Test MSE 0.003001442874428272 Test RE 0.02618623742810

45 Train Loss 1.6269069 Test MSE 2.1141514980578524 Test RE 0.6949860908325453
46 Train Loss 1.4486172 Test MSE 1.850913489866039 Test RE 0.650281083397356
47 Train Loss 1.2727199 Test MSE 1.6474285517890181 Test RE 0.6134954618940927
48 Train Loss 0.89343077 Test MSE 0.8139749061732765 Test RE 0.43123456958042
49 Train Loss 0.58941126 Test MSE 0.4367101363691966 Test RE 0.315867320604644
50 Train Loss 0.33998436 Test MSE 0.05570233927393808 Test RE 0.1128092670120036
51 Train Loss 0.22706114 Test MSE 0.04443543352065347 Test RE 0.10075642997403268
52 Train Loss 0.1487733 Test MSE 0.021253168061630447 Test RE 0.06968188628881918
53 Train Loss 0.10465523 Test MSE 0.020359634780018038 Test RE 0.06820136265042524
54 Train Loss 0.07072458 Test MSE 0.009397938684749838 Test RE 0.04633662135046549
55 Train Loss 0.048520286 Test MSE 0.006071872172734701 Test RE 0.037245119351949775
56 Train Loss 0.040520042 Test MSE 0.0051155027257525 Test RE 0.03418630952870471
57 Train Loss 0.032948345 Test

36 Train Loss 5.6320004 Test MSE 1.4975625232579757 Test RE 0.5849254823940512
37 Train Loss 3.9521878 Test MSE 0.5883671281330741 Test RE 0.36663359788869276
38 Train Loss 3.0538273 Test MSE 0.37648575239819154 Test RE 0.29327992263561026
39 Train Loss 2.368588 Test MSE 0.2619781665512063 Test RE 0.244647402730933
40 Train Loss 1.8839486 Test MSE 0.16670128008540863 Test RE 0.19515403162256031
41 Train Loss 1.5317867 Test MSE 0.13904725432626255 Test RE 0.17823347273685547
42 Train Loss 1.2079184 Test MSE 0.1034473414379887 Test RE 0.15373322124006447
43 Train Loss 1.0010798 Test MSE 0.08244202648652846 Test RE 0.13724053094171348
44 Train Loss 0.8127874 Test MSE 0.07363144183313057 Test RE 0.1296999213370123
45 Train Loss 0.6765146 Test MSE 0.04790460200119007 Test RE 0.10461565498166948
46 Train Loss 0.5304737 Test MSE 0.049043636572784995 Test RE 0.1058520791931957
47 Train Loss 0.4485244 Test MSE 0.0430916046084517 Test RE 0.0992211813222075
48 Train Loss 0.33732837 Test MSE 0.035

In [13]:
label

'KG_atanh'

In [14]:
import scipy.io as sio

In [15]:
for tune_reps in range(25):
    if tune_reps not in s:
        label = "KG_stan_tune"+str(tune_reps)+".mat"
        data = sio.loadmat(label)
        re = np.array(data["test_re_loss"])
        print(tune_reps," ",np.mean(re[:,-1]))

NameError: name 's' is not defined

In [16]:
a = 0
for i in range(10):
    a = a+ test_re_full[i][-1]
    print(test_re_full[i][-1])
    
print("a = ",a/10)

0.019008550524342072
0.07143034103102627
0.012902022197801778
0.8798929221233429
0.030078128935536574
1.3600168406256998
0.01890595458791737
0.014575639134573127
0.007159216581810301
0.02469968809965618
a =  0.24386693038417065
